In [1]:
from torch_geometric.datasets import Planetoid, Amazon, TUDataset, Reddit, AmazonProducts, WikipediaNetwork, Wikidata5M, Reddit2
from ogb.nodeproppred import PygNodePropPredDataset
import torch_geometric.transforms as T

# dataset0 = Reddit(root='../datasets/Reddit', transform=T.ToSparseTensor())
# data0 = dataset0[0].to('cpu')

In [1]:
from dataset_loader import loader_dict

dataset, data = loader_dict['amazon'](adj_t=True)

In [9]:
data.y[:,0].avg()

AttributeError: 'Tensor' object has no attribute 'avg'

In [2]:
dataset = AmazonProducts(root=f'../datasets/AmazonProducts', transform=T.ToSparseTensor())
data = dataset[0].to('cpu')


In [ ]:
dataset = PygNodePropPredDataset(name='ogbn-proteins', root=f'../datasets/ogbn-proteins')
data = dataset[0].to(device)

In [32]:
# dataset = AmazonProducts(root=f'../datasets/AmazonProducts')
dataset = PygNodePropPredDataset(name='ogbn-proteins', root=f'../datasets/ogbn-proteins')

In [33]:
data = dataset[0].to('cpu')

In [11]:
data.y.sum(axis=1).max()

tensor(7)

In [44]:
import torch 

def multihot2decimal(arr):
    idx = 0
    value_mapper = {}
    new_y = []
    for i in arr:
        hash_val = hashlib.sha256(i.numpy().tobytes()).hexdigest()
        if hash_val not in value_mapper:
            value_mapper[hash_val] = idx
            idx += 1    
        new_y.append(value_mapper[hash_val])

    return torch.tensor(new_y), idx - 1


In [45]:
y_, num_classes = multihot2decimal(data.y)

In [46]:
num_classes

31994

In [38]:
data.y.shape

torch.Size([132534, 112])

In [40]:
max(y_)

tensor(31994)

In [36]:
len(y_)

132534

In [20]:
import hashlib

values = set()
for i in data.y:
    values.add(hashlib.sha256(i.numpy().tobytes()).hexdigest())

len(values)

386

In [ ]:
values

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()

# Fit and transform the multihot encoded feature vectors
integer_classes = label_binarizer.fit_transform(data.y)

# Convert binary arrays to decimal classes
decimal_classes = [int(''.join(map(str, binary_array)), 2) for binary_array in integer_classes]

decimal_classes

torch.Size([232965, 608])

In [12]:
import scipy.sparse as sparse


# from scipy.io import mmwrite

# mmwrite("ogbn_protein.mtx", b.adj_t.to_dense().numpy())

In [6]:
import fast_matrix_market as fmm

def dataset_to_mtx(dataset, filename):
    coo = dataset.adj_t.to_torch_sparse_coo_tensor()
    coo = coo.coalesce()
    i = coo.indices()[0] 
    j = coo.indices()[1]
    v = coo.values()
    shape = coo.size()
    coo_sci = sparse.coo_matrix((v,(i,j)),shape=(shape[0], shape[1]))
    fmm.mmwrite(filename,coo_sci)


In [16]:
dataset = PygNodePropPredDataset(name='ogbn-proteins', root='../datasets/ogbn-proteins')
data = dataset[0].to('cpu')

In [17]:
data

Data(num_nodes=132534, edge_index=[2, 79122504], edge_attr=[79122504, 8], node_species=[132534, 1], y=[132534, 112])

In [13]:
# from torch_geometric.transforms import ToDense
# adj_d = data.adj_t.to_dense()
# adj_d @ data.edge_attr

In [18]:
data.edge_index

tensor([[    1, 96401,     1,  ..., 86562, 86562, 86435],
        [96401,     1, 68343,  ..., 86404, 86435, 86562]])

In [19]:
src, dest = data.edge_index
indices = dest

In [6]:
import torch
from torch_scatter import scatter_add

N = 3
indices = torch.tensor([1, 2, 0])

scatter_add(torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), indices, dim=0, dim_size=N)
    

tensor([[7, 8, 9],
        [1, 2, 3],
        [4, 5, 6]])

In [22]:
data

Data(num_nodes=132534, edge_index=[2, 79122504], edge_attr=[79122504, 8], node_species=[132534, 1], y=[132534, 112])

In [21]:
node_features_1hop = scatter_add(data.edge_attr, indices, dim=0, dim_size=num_nodes)
node_features_1hop.shape

torch.Size([132534, 8])

# Reddit2

In [6]:
dataset = Reddit2(root='../datasets/Reddit2', transform=T.ToSparseTensor())
data = dataset[0].to('cpu')
data

Processing...
Done!


Data(x=[232965, 602], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965], adj_t=[232965, 232965, nnz=23213838])

In [5]:
data.x = data.adj_t.mean(dim=1).reshape([-1, 1])

In [7]:
data.x

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

In [ ]:
# OGBN-Protein

dataset = PygNodePropPredDataset(name='ogbn-proteins',
                                     transform=T.ToSparseTensor())
data = dataset[0]

# Move edge features to node features.
data.x = data.adj_t.mean(dim=1).reshape([-1, 1])
data.adj_t.set_value_(None)

split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

if args.method == 'gcn':
model = GCN(data.num_features, args.hidden_channels, 112,
            args.num_layers, args.dropout).to(device)
elif args.method == 'gcnjk':
model = GCNJK(data.num_features, args.hidden_channels, 112,
            args.num_layers, args.dropout).to(device)
else:
raise ValueError('invalid method, use gcn or gcnjk')

# Pre-compute GCN normalization.
adj_t = data.adj_t.set_diag()
deg = adj_t.sum(dim=1).to(torch.float)
deg_inv_sqrt = deg.pow(-0.5)
deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
adj_t = deg_inv_sqrt.view(-1, 1) * adj_t * deg_inv_sqrt.view(1, -1)
data.adj_t = adj_t


# Mag

In [2]:
# from tests.cpu.dataset_loader import load_ogbn_protein
from dataset_loader import *

a, b = load_ogbn_protein()

In [5]:
b.edge_index

In [4]:
b.edge_attr.shape

torch.Size([79122504, 8])

In [26]:
print(a.num_node_features_, a.num_classes,  a.num_classes_)
print(b)

200 107 107
Data(x=[1569960, 200], y=[1569960], train_mask=[1569960], val_mask=[1569960], test_mask=[1569960], adj_t=[1569960, 1569960, nnz=264339468])


In [27]:
len(set({int(i) for i in b.y}))

8

In [28]:
set({int(i) for i in b.y})

{0, 1, 2, 3, 4, 5, 6, 7}

In [3]:
dataset = PygNodePropPredDataset(name='ogbn-mag', root='../datasets/ogbn-mag')
data = dataset[0].to('cpu')

In [ ]:
dataset.num_node_features = 128

In [7]:
data.num_nodes = data.num_nodes_dict['paper']
data.x = data.x_dict['paper']
data.y =  data.y_dict['paper']


In [ ]:
dataset.get_idx_split()

In [10]:
N = data.num_nodes
import torch
data.train_mask = torch.zeros(N, dtype=torch.bool)
data.train_mask[dataset.get_idx_split()['train']['paper']] = True

data.test_mask = torch.zeros(N, dtype=torch.bool)
data.test_mask[dataset.get_idx_split()['test']['paper']] = True

data.valid_mask = torch.zeros(N, dtype=torch.bool)
data.valid_mask[dataset.get_idx_split()['valid']['paper']] = True

In [13]:
data.y = torch.squeeze(data.y)

In [17]:
data.edge_index = data.edge_index_dict[('paper', 'cites', 'paper')]

In [18]:
data = T.ToSparseTensor()(data)

In [ ]:
data

In [22]:
data.x.shape[1]

128

In [20]:
dataset.num_node_features = data.x.shape[1]

0

In [21]:
dataset.num_classes

349

# Amazon Products

In [12]:
# dataset = Amazon("datasets/Amazon",name='computers' ,transform=T.Compose([T.ToSparseTensor(),T.RandomNodeSplit()]))
dataset = AmazonProducts(root='../datasets/AmazonProducts')
# dataset = TUDataset(root='../datasets/AmazonProducts', name='AmazonProducts', transform=T.ToSparseTensor())
# dataset = Wikidata5M(root='../datasets/Wikidata5M', transform=T.ToSparseTensor())

data = dataset[0].to('cpu')

In [10]:
data.y = data.y.sum(axis=1)

In [ ]:
dataset.num_classes
dataset.num_node_features

# Product

In [3]:
# dataset = PygNodePropPredDataset(name='ogbn-mag', root='../datasets/ogbn-mag', transform=T.ToSparseTensor())
dataset = PygNodePropPredDataset(name='ogbn-products', root='../datasets/ogbn-products')
data = dataset[0].to('cpu')

In [5]:
data

Data(num_nodes=2449029, edge_index=[2, 123718280], x=[2449029, 100], y=[2449029, 1])

In [11]:
import torch
data.y = torch.squeeze(data.y)

In [9]:
N = data.num_nodes
import torch
data.train_mask = torch.zeros(N, dtype=torch.bool)
data.train_mask[dataset.get_idx_split()['train']] = True

data.test_mask = torch.zeros(N, dtype=torch.bool)
data.test_mask[dataset.get_idx_split()['test']] = True

data.valid_mask = torch.zeros(N, dtype=torch.bool)
data.valid_mask[dataset.get_idx_split()['valid']] = True

In [12]:
data

Data(num_nodes=2449029, edge_index=[2, 123718280], x=[2449029, 100], y=[2449029], train_mask=[2449029], test_mask=[2449029], valid_mask=[2449029])

In [13]:
data = T.ToSparseTensor()(data)

data

Data(num_nodes=2449029, x=[2449029, 100], y=[2449029], train_mask=[2449029], test_mask=[2449029], valid_mask=[2449029], adj_t=[2449029, 2449029, nnz=123718280])

# Protein

In [11]:

# dataset = PygNodePropPredDataset(name='ogbn-mag', root='../datasets/ogbn-mag', transform=T.ToSparseTensor())
dataset = PygNodePropPredDataset(name='ogbn-proteins', root='../datasets/ogbn-proteins')
data = dataset[0].to('cpu')

In [13]:
data.num_classes

AttributeError: 'GlobalStorage' object has no attribute 'num_classes'

In [4]:
N = data.num_nodes
import torch
data.train_mask = torch.zeros(N, dtype=torch.bool)
data.train_mask[dataset.get_idx_split()['train']] = True

data.test_mask = torch.zeros(N, dtype=torch.bool)
data.test_mask[dataset.get_idx_split()['test']] = True

data.valid_mask = torch.zeros(N, dtype=torch.bool)
data.valid_mask[dataset.get_idx_split()['valid']] = True

In [35]:
data

Data(num_nodes=132534, edge_index=[2, 79122504], edge_attr=[79122504, 8], node_species=[132534, 1], y=[132534, 112], train_mask=[132534], test_mask=[132534], valid_mask=[132534])

In [5]:
data.y = data.y.sum(axis=1)

In [37]:
data

Data(num_nodes=132534, edge_index=[2, 79122504], edge_attr=[79122504, 8], node_species=[132534, 1], y=[132534], train_mask=[132534], test_mask=[132534], valid_mask=[132534])

In [6]:
data.num_nodes

132534